# Iceland Snow and Ice Monitoring

This notebook implements a workflow for monitoring snow and ice in Iceland using Sentinel-2 data via the EOPF Zarr format.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from xcube.core.store import new_data_store
from xcube_eopf.utils import reproject_bbox
from shapely.geometry import box
import shapely.geometry

# Initialize Data Store
store = new_data_store("eopf-zarr")

## Seeds
Load the glacier seeds (points) and define the Area of Interest.

In [ ]:
# Load seeds
seeds_gdf = gpd.read_file("data/Iceland_Seeds.geojson")

# Reproject to WGS84 for search
seeds_gdf = seeds_gdf.to_crs("EPSG:4326")

# Create a buffer around seeds (approx 5km for 10x10km tile) or just use bounds
# For the native algorithm, we want scenes covering these seeds.
total_bounds = seeds_gdf.total_bounds # [minx, miny, maxx, maxy]
print(f"Total Bounds (EPSG:4326): {total_bounds}")

## Sentinel Native Algorithm

This algorithm simulates a file-based workflow by processing full Sentinel-2 scenes (Sentinel-2 L2A) retrieved from the EOPF Zarr store. 
It avoids tile-based optimization and instead loads the full scene extent to compute NDSI and classify snow.

In [ ]:
def sentinel_native_algorithm(seeds_gdf, time_range, store):

    bbox = list(seeds_gdf.total_bounds)
    
    # Using UTM 27N for Iceland (EPSG:32627) as a reasonable projection for the ROI
    crs = "EPSG:32627"
    try:
        bbox_utm = reproject_bbox(bbox, "EPSG:4326", crs)
    except Exception as e:
        print(f"Warning: Bbox reprojection failed ({e}). Using original bbox.", flush=True)
        bbox_utm = bbox
        crs = "EPSG:4326" # Fallback

    print(f"Opening datacube for {time_range} over {bbox}...", flush=True)
    
    try:
        # Open the dataset as a cube. 
        # This represents the "file" access in this simulated native algorithm 
        # because we will force-load the entire spatial slice for each time step.
        ds = store.open_data(
            data_id="sentinel-2-l2a",
            bbox=bbox_utm,
            time_range=time_range,
            spatial_res=60, # Using 60m to be faster for the prototype/demo, change to 10 for full res
            crs=crs,
            variables=["b03", "b11", "scl"]
        )
    except Exception as e:
        print(f"Failed to open data: {e}", flush=True)
        return []
    
    if "time" not in ds.coords or ds.time.size == 0:
        print("No data found for the given criteria.", flush=True)
        return []

    print(f"Found {ds.time.size} time steps. Processing...", flush=True)
    
    results = []
    for t in ds.time.values:
        t_str = str(t)
        print(f"Processing time: {t_str}", flush=True)
        
        try:
            # Select the time slice
            ds_slice = ds.sel(time=t)
            
            # "Simulate Native": Load the FULL slice into memory.
            # In a real file-based workflow, this equates to reading the file from disk/network.
            print("  Loading full slice (simulating file download)...", flush=True)
            ds_slice = ds_slice.compute() 
            
            # Check for SCL
            if "scl" in ds_slice:
                valid_mask = ~ds_slice["scl"].isin([3, 8, 9])
                ds_valid = ds_slice.where(valid_mask)
            else:
                ds_valid = ds_slice
                valid_mask = (ds_slice["b03"] > 0) # Fallback valid mask
            
            # NDSI
            if "b03" in ds_valid and "b11" in ds_valid:
                green = ds_valid["b03"]
                swir = ds_valid["b11"]
                
                # Avoid division by zero
                denom = (green + swir)
                ndsi = (green - swir) / denom.where(denom != 0)
                
                snow_map = ndsi > 0.42
                
                # Stats
                valid_count = valid_mask.sum().item()
                snow_count = snow_map.where(valid_mask).sum().item()
                
                snow_pct = (snow_count / valid_count * 100) if valid_count > 0 else 0
                print(f"  Snow Cover: {snow_pct:.2f}%", flush=True)
                
                results.append({
                    "time": t_str,
                    "snow_pct": snow_pct,
                    "snow_area_px": snow_count
                })
                
                if snow_pct > 30:
                    print("  > 30% Snow. (Native algorithm would check neighbors here)", flush=True)
            
        except Exception as e:
            print(f"  Error processing slice {t_str}: {e}", flush=True)
            
    return results

# Example Usage
time_range = ["2025-06-01", "2025-06-30"]
results = sentinel_native_algorithm(seeds_gdf, time_range, store)

## Comparison
Compare the native results with the tile-aware Zarr approach.